In [ ]:
import numpy as np
import pandas as pd
import random

function to compute the entropy of a leaf

In [ ]:
def entropy(freq):
    freq_0 = freq[np.array(freq).nonzero()[0]]
    prob_0 = freq_0/float(freq_0.sum())
    return -np.sum(prob_0*np.log2(prob_0))

a class that represent each node of the tree

every node of the tree will be a instance of this class

In [ ]:
class TreeNode(object):
    def __init__(self, ids =[], children = [], entropy = 0, depth = 0):
        self.ids = ids           
        self.entropy = entropy  
        self.depth = depth      
        self.split_attribute = None 
        self.children = children 
        self.order = None     
        self.label = None  

    def __repr__(self):
        if self.split_attribute!=None :return self.split_attribute   
        else: return 'leaf'
    
    def set_properties(self, split_attribute, order):
        self.split_attribute = split_attribute
        self.order = order

    def set_label(self, label):
        self.label = label

Decision Tree class that have all attributes and functions of the tree 

with this class we can fit data to make an appropriate tree , evaluate and even prun it

In [ ]:
class DecisionTreeID3(object):
    def __init__(self, max_depth= 10):
        self.root = None 
        self.max_depth = max_depth
        self.Ntrain = 0 
        self.nodes=[] 
    
    def fit(self, data, target, x_val, y_val):
        self.Ntrain = data.count()[0]  
        self.data = data              
        self.attributes = list(data)  
        self.target = target         
        self.labels = target.unique()
        
        ids = range(self.Ntrain)
        self.root = TreeNode(ids = ids, entropy = self._entropy(ids), depth = 0)  
        queue = [self.root]
        while queue: 
            node = queue.pop()

            if node.depth < self.max_depth:
                node.children = self._split(node)
                if not node.children: 
                    self._set_label(node)
                queue += node.children
                self.nodes += node.children
            else:
                self._set_label(node)       
        self.acc=self.evaluation(x_val,y_val,True)      
                
    def _entropy(self, ids):
        if len(ids) == 0: return 0
        ids=list(ids)
        freq = np.array(self.target[ids].value_counts())
        return entropy(freq)

    def _set_label(self, node):
        target_ids =list(node.ids)
        node.set_label(self.target[target_ids].mode()[0])

    
    def _split(self, node):
        ids = node.ids 
        best_gain = 0
        best_splits = []
        best_attribute = None
        order = None
        sub_data = self.data.iloc[ids, :]
        for i, att in enumerate(self.attributes):
            values = self.data.iloc[ids, i].unique().tolist()
            if len(values) == 1: continue # entropy = 0
            splits = []
            for val in values: 
                sub_ids = sub_data.index[sub_data[att] == val].tolist()
                splits.append([sub_id for sub_id in sub_ids])
                HxS= 0
            for split in splits:
                HxS += len(split)*self._entropy(split)/len(ids)
            gain = node.entropy - HxS 
            if gain > best_gain:
                best_gain = gain 
                best_splits = splits
                best_attribute = att
                order = values
        node.set_properties(best_attribute, order)
        child_nodes = [TreeNode(ids = split,
                     entropy = self._entropy(split), depth = node.depth + 1) for split in best_splits]
        return child_nodes

    def predict(self, new_data):

        npoints = new_data.count()[0]
        labels = [None]*npoints
        for n in range(npoints):
            x = new_data.iloc[n, :] 
            node = self.root
            while node.children: 
                if x[node.split_attribute] in node.order:
                    node = node.children[node.order.index(x[node.split_attribute])]
                else:           
                    node.label=self._set_label(node)
                    break
            labels[n] = node.label  
        return labels
    
    def evaluation(self,x_test,y_test,show_acc=False):
        z=self.predict(x_test)
        count=0
        for i in range(len(y_test)):
            if y_test[i]==z[i]:
                count+=1
        if show_acc:
            print((count/len(x_test))*100) 
        self.acc=count/len(x_test)*100
        return count/len(x_test)*100
    
    def _eval(self,x_test,y_test,show_acc=False):
        z=self.predict(x_test)
        count=0
        for i in range(len(y_test)):
            if y_test[i]==z[i]:
                count+=1
        if show_acc:
            print((count/len(x_test))*100) 
        return count/len(x_test)*100    
    
    
    def pruning(self,x2,y2,x_tr,y_tr,x_te,y_te): 
        max_depth=0
        for n in self.nodes:
            if n.depth>max_depth:
                max_depth=n.depth
        prun_list=[]
        for n in self.nodes:
            if n.children:
                prun_list.append(n)
        prun_list1=[]
        for dep in range(1,max_depth):
            for n in prun_list:
                if n.depth==dep: prun_list1.append(n)   
        before_pr=len(prun_list1)
        acc=100
        last_acc=self.acc
        nodes=[]
        val_acc=[]
        while acc +.2 >= last_acc: 
            pr_node=prun_list1.pop()
            if pr_node.children:
                last_node=pr_node.children
                pr_node.children=[]
                self._set_label(pr_node)
                last_acc=self.acc
                acc=self.evaluation(x2,y2,False)
                before_pr-=1
                nodes.append(before_pr)
                print(before_pr)
                val_acc.append(self._eval(x2,y2,True))
                print('    ')
                print('    ')
        pr_node.children=last_node        
        self.acc=self.evaluation(x2,y2)
        
        return nodes,val_acc


some preprocessing for the adult data

In [ ]:
def load_data(add):
    data=pd.read_csv(add)
    data=pd.DataFrame(data)
    data=data.replace({'<=50K':0})
    data=data.replace({'>50K':1})
    return data

In [ ]:
def pre_train(train_data,percentage):
    idx=range(len(train_data))
    idx=list(idx)
    random.shuffle(idx)
    aa=int(percentage*len(train_data)/100)
    idx=idx[:aa]
    x1=train_data.iloc[idx,:]
    x1.index=range(aa)
    x = x1.iloc[:, 1:]
    y = x1.iloc[:, 0] 
    return x,y

In [ ]:
def train_val_split(data,percentage):
    idx=range(len(data))
    idx=list(idx)
    random.shuffle(idx)
    aa=int(percentage*len(data)/100)
    idx1=idx[:aa]
    idx2=idx[aa:]
    x_tr=data.iloc[idx1,:]
    x_val=data.iloc[idx2,:]
    x_tr.index=range(aa)
    x_val.index=range(len(data)-aa)
    x_tr_ = x_tr.iloc[:, 1:]
    y_tr_ = x_tr.iloc[:, 0] 
    x_val_ = x_val.iloc[:, 1:]
    y_val_= x_val.iloc[:, 0] 
    
    return x_tr_,y_tr_,x_val_,y_val_   

In [ ]:
data=load_data('C:/Users/NP-soft/Desktop/data/Decision Tree ID3/adult.train.csv')
x1,y1,x2,y2=train_val_split(data,80)
x1,y1=pre_train(data,100)
test_data=load_data('C:/Users/NP-soft/Desktop/data/Decision Tree ID3/adult.test.csv')
x2,y2,x_test,y_test=train_val_split(test_data,25)
x_test = test_data.iloc[:, 1:]
y_test= test_data.iloc[:, 0] 

make a instance of DecisionTreeID3 class , fit the adult data to it and show the evlauation and test accuracy 

In [ ]:
tree = DecisionTreeID3(max_depth = 7)
tree.fit(x1,y1,x2,y2)
tree.evaluation(x_test,y_test)

76.64


77.13